# Eat Safe, Love

## Notebook Set Up

In [12]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(mongo.list_database_names())

['EPA', 'admin', 'autosauras', 'classDB', 'class_db', 'config', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'test', 'travel_db', 'uk_food']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [18]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene':20}
fields = {"BusinessName":1,"scores.Hygiene":1}
# Use count_documents to display the number of documents in the result
query_count = establishments.count_documents(query)

# Display the first document in the results using pprint
pprint(db.establishments.find_one(query, fields))

{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('6497a39580b695e9e5e11692'),
 'scores': {'Hygiene': 20}}


In [21]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query, fields))
hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("The number of rows in the DataFrame is ", len(results))
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

The number of rows in the DataFrame is  41


,_id,BusinessName,scores
0,6497a39580b695e9e5e11692,The Chase Rest Home,{'Hygiene': 20}
1,6497a39580b695e9e5e11a13,Brenalwood,{'Hygiene': 20}
2,6497a39580b695e9e5e11d1d,Melrose Hotel,{'Hygiene': 20}
3,6497a39580b695e9e5e11f0b,Seaford Pizza,{'Hygiene': 20}
4,6497a39580b695e9e5e11f1b,Golden Palace,{'Hygiene': 20}
5,6497a39580b695e9e5e128b6,Ashby's Butchers,{'Hygiene': 20}
6,6497a39580b695e9e5e12ada,South Sea Express Cuisine,{'Hygiene': 20}
7,6497a39580b695e9e5e14003,Golden Palace,{'Hygiene': 20}
8,6497a39580b695e9e5e14449,The Tulip Tree,{'Hygiene': 20}
9,6497a39580b695e9e5e14c59,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query =

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 
longitude = 

query = 
sort =  

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
